In [1]:
%matplotlib inline
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplot
from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import train_test_split


from keras.utils import *
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.preprocessing import image
from keras.optimizers import RMSprop,SGD
from keras.layers.normalization import BatchNormalization

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [47]:
train_data = json.load(open('/home/ubuntu/data/iceberg/train.json'))
print type(train_data)
print len(train_data)
print train_data[0].keys()

<type 'list'>
1604
[u'is_iceberg', u'inc_angle', u'band_2', u'id', u'band_1']


In [48]:
test_data = json.load(open('/home/ubuntu/data/iceberg/test.json'))
print type(test_data)
print len(test_data)
print test_data[0].keys()

<type 'list'>
8424
[u'inc_angle', u'band_2', u'id', u'band_1']


# Approach 1: Using convolution NN

In [50]:
#test_data = json.load(open('/home/ubuntu/data/iceberg/test.json'))
#print type(test_data)
#print len(test_data)
#print test_data[0].keys()
#train_data = np.random.random(train_data)

In [51]:
train_X = np.array([np.array([np.array(d['band_1']).reshape(75,75),np.array(d['band_2']).reshape(75,75),
                             np.array(d['band_1']).reshape(75,75)/np.array(d['band_2']).reshape(75,75)]) \
                    for d in train_data])
ids = np.array([d['id'] for d in train_data])
train_y = np.array([d['is_iceberg'] for d in train_data])
del train_data

In [52]:
#train_X,valid_X,train_y,valid_y = train_test_split(train_X,train_y,
#                                                   train_size=0.75,
#                                                   random_state=123,stratify=train_y)
total_samples = train_X.shape[0]
#train_X = np.random.permutation(train_X)
#train_y = np.random.permutation(train_y)
train_X1 = train_X[:int(total_samples*0.75)]
train_y1 = train_y[:int(total_samples*0.75)]
valid_X = train_X[int(total_samples*0.75):]
valid_y = train_y[int(total_samples*0.75):]


print train_X1.shape,valid_X.shape,train_y1.shape,valid_y.shape

(1203, 3, 75, 75) (401, 3, 75, 75) (1203,) (401,)


In [53]:
print train_X1.shape,train_y1.shape,ids.shape
train_y1 = np_utils.to_categorical(train_y1)
valid_y = np_utils.to_categorical(valid_y)
del train_y,train_X
print train_y1.shape,valid_y.shape
#print train_y_one_hot_encoded[:2]
#train_X[:1],train_y[:5],ids[:5]

(1203, 3, 75, 75) (1203,) (1604,)
(1203, 2) (401, 2)


In [142]:
def create_visualiization():
    fig = mplot.figure(figsize=(12,6))
    for i,y in zip(range(12),train_y_one_hot_encoded[:12]):
        fig.add_subplot(3,4,i+1)
        mplot.text(0,0,str(y))

        mplot.imshow(train_X[i,0],interpolation='none')

        mplot.xticks([])
        mplot.yticks([])
        mplot.yticks([])
#mplot.subplot()

In [54]:
import vgg16bn
reload(vgg16bn)

<module 'vgg16bn' from 'vgg16bn.pyc'>

In [55]:
vgg = vgg16bn.Vgg16BN(size=(75,75),include_top=True)

In [56]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_8 (Lambda)                (None, 3, 75, 75)     0           lambda_input_8[0][0]             
____________________________________________________________________________________________________
zeropadding2d_74 (ZeroPadding2D) (None, 3, 77, 77)     0           lambda_8[0][0]                   
____________________________________________________________________________________________________
convolution2d_74 (Convolution2D) (None, 64, 75, 75)    1792        zeropadding2d_74[0][0]           
____________________________________________________________________________________________________
zeropadding2d_75 (ZeroPadding2D) (None, 64, 77, 77)    0           convolution2d_74[0][0]           
___________________________________________________________________________________________

In [57]:
vgg.ft(2)

In [60]:
layers = vgg.model.layers
#for layer in layers: print layer.trainable
for layer in layers: layer.trainable = True
#for layer in layers: print layer.trainable

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [62]:
vgg.model.fit(train_X1,train_y1,validation_data=(valid_X,valid_y),nb_epoch=3)

Train on 1203 samples, validate on 401 samples
Epoch 1/3
1203/1203 [==============================] - 3s - loss: 0.6992 - acc: 0.7149 - val_loss: 1.0685 - val_acc: 0.3591
Epoch 2/3
1203/1203 [==============================] - 3s - loss: 0.6826 - acc: 0.7182 - val_loss: 1.0571 - val_acc: 0.3591
Epoch 3/3
1203/1203 [==============================] - 3s - loss: 0.7160 - acc: 0.7066 - val_loss: 1.1318 - val_acc: 0.3591


In [33]:

flat_idx = [idx for idx,layer in enumerate(layers) if type(layer) is Flatten][0]
for layer in layers[flat_idx+1:]:layer.trainable=True

In [34]:
vgg.model.fit(train_X1,train_y1,validation_data=(valid_X,valid_y),nb_epoch=3)

Train on 1203 samples, validate on 401 samples
Epoch 1/3
1203/1203 [==============================] - 3s - loss: 0.7841 - acc: 0.6833 - val_loss: 1.0373 - val_acc: 0.3591
Epoch 2/3
1203/1203 [==============================] - 3s - loss: 0.7206 - acc: 0.7066 - val_loss: 1.2963 - val_acc: 0.3566
Epoch 3/3
1203/1203 [==============================] - 3s - loss: 0.6693 - acc: 0.7232 - val_loss: 1.0674 - val_acc: 0.3591


In [46]:
#vgg.model.optimizer.lr=0.00001
print vgg.model.optimizer.lr
#for layer in layers: layer.trainable=True
vgg.model.fit(train_X1,train_y1,validation_data=(valid_X,valid_y),nb_epoch=5)

1e-05
Train on 1203 samples, validate on 401 samples
Epoch 1/5
1203/1203 [==============================] - 3s - loss: 0.6488 - acc: 0.7332 - val_loss: 1.0235 - val_acc: 0.3915
Epoch 2/5
1203/1203 [==============================] - 3s - loss: 0.6247 - acc: 0.7556 - val_loss: 1.5032 - val_acc: 0.3641
Epoch 3/5
1203/1203 [==============================] - 3s - loss: 0.6624 - acc: 0.7215 - val_loss: 1.0421 - val_acc: 0.3815
Epoch 4/5
1203/1203 [==============================] - 3s - loss: 0.6191 - acc: 0.7481 - val_loss: 0.8492 - val_acc: 0.4090
Epoch 5/5
1203/1203 [==============================] - 3s - loss: 0.5958 - acc: 0.7581 - val_loss: 0.9131 - val_acc: 0.3940


# Approach 2: KNeighborsClassifier

In [145]:
import sklearn

In [147]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [151]:
train_X = []
for row in train_data:
    train_X.append(np.array(row['band_1']))
    train_X.append(np.array(row['band_2']))
train_X = np.array(train_X)

train_y = []
for row in train_data:
    train_y.append(np.array(row['is_iceberg']))
    train_y.append(np.array(row['is_iceberg']))
train_y = np.array(train_y)

print train_X.shape,train_y.shape

test_X = []
for row in test_data:
    test_X.append(np.array(row['band_1']))
    test_X.append(np.array(row['band_2']))
test_X = np.array(test_X)

print test_X.shape

labels = []
for row in test_data:
    labels.append(np.array(row['id']))
labels = np.array(labels)

print labels.shape

 (3208, 5625) (3208,)


In [154]:
samples=int(train_X.shape[0]*.75)
train_X1 = train_X[:samples]
train_y1 = train_y[:samples]

valid_X1 = train_X[samples:]
valid_y1 = train_y[samples:]

print train_X1.shape,train_y1.shape,valid_X1.shape,valid_y1.shape

 (2406, 5625) (2406,) (802, 5625) (802,)


In [155]:
knc = KNeighborsClassifier()
sc = StandardScaler()

sc.fit(train_X1)
train_X1 = sc.transform(train_X1)
test_X = sc.transform(test_X)

In [156]:
knc.fit(train_X1,train_y1)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [158]:
knc.score(valid_X1,valid_y1)

0.62842892768079806

In [ ]:
probs = knc.predict_proba(test_X)

In [ ]:
prob_final = (probs[::2]+probs[1::2])/len(probs.shape[0]/2)